### Importing librairies

In [72]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint
import pandas as pd

### API connection

In [ ]:
c_id = getpass.getpass()
c_se = getpass.getpass()

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se))

### Get maximum of track uri

Mise en forme des données : 2 tables : 
- une avec les infos du son (titre, allbum, artist name...)
- une avec les données de l'analyse
Les deux liés par l'URI

On va récupérer un maximum d'URI à l'aide d'une query sur les styles dans un permier temps. Ensuite nous complèterons avec l'aide des recommandations et des playlistes.

In [ ]:
musical_styles = ["Rock", "Pop", "Hip-Hop", "Rap", "R&B", "Country", "Electronic", "Jazz", "Classical", "Blues", "Reggae", "Funk", "Indie", "Metal", "Punk", "Folk", "Dance", "World", "Latin", "Alternative", "Gospel", "Soul", "Disco", "House", "Techno", "EDM", "Dubstep", "Trance", "Chillout", "Ambient", "Funk", "Jazz", "Swing", "Country", "Bluegrass", "Fusion", "Latin Jazz", "Big Band", "Symphonic", "Chamber", "Orchestral", "Opera", "Baroque", "Choral", "Renaissance", "Acoustic", "Delta Blues", "Chicago Blues", "Soul Blues", "Boogie Woogie", "Piano Blues", "Dub", "Ska", "Rocksteady", "Soca", "Calypso", "Merengue", "Salsa", "Cumbia", "Tango", "Bachata", "Flamenco", "Mariachi", "Sertanejo", "Forró", "Pagode", "Fado", "K-Pop", "J-Pop", "C-Pop", "Reggaeton", "Trap", "Hardstyle", "Grime", "Electroswing", "Breakbeat", "IDM", "Downtempo", "Industrial", "Nu-Metal", "Grunge", "Death Metal", "Thrash Metal", "Progressive Metal", "Post-Rock", "Shoegaze", "Post-Punk", "Psychedelic", "Post-Metal", "Experimental", "Neoclassical", "Soundtrack", "New Age", "Celtic", "Gothic", "Fusion", "Worldbeat", "Bollywood", "Funk", "Raggacore", "Trip Hop"]

In [106]:
query = "genre:'rock'"
search_results = sp.search(q=query, type='playlist', limit=50)  #track_info.keys() retourne pas plus d'info

In [ ]:
def get_playlist_id() # musical_styles as input, and output dict : {"musical_syle": [id1,id2]}

In [ ]:
def get_tracks_id_from_playlist_id() : #return a Pandas dataframe with columns : "track_id	track_uri	track_name	track_artists	track_preview_url	track_popularity	album_id	album_name	album_uri	album_ release_date"

In [115]:
search_results = sp.category_playlists(category_id= 'rock')

In [131]:
search_results["playlists"]["items"][3]["external_urls"]

{'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX8HxJlVUR3HN'}

In [123]:
search_results["playlists"]["items"][0].keys()

dict_keys(['collaborative', 'description', 'external_urls', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [88]:
search_results["tracks"].keys() #Nested dictionnary
pprint.pprint(search_results["tracks"]["items"][0]["popularity"]) # On est interessé par : 'album'(-> 'name', 'artist'->'name'), 'artists','duration_ms','id','uri','preview_url','explicit','name','popularity'

KeyError: 'tracks'

### Get audio feautures from tracks URI

In [64]:
# get audio features, et audio analysis
sp.audio_analysis("spotify:track:2QjOHCTQ1Jl3zawyYOpxh6") #Too much
sp.audio_features("spotify:track:2QjOHCTQ1Jl3zawyYOpxh6")[0].keys() #Ce qu'on va utiliser

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [ ]:
# for style in musical_styles : 
#     query = "genre:'{}'".format(style)
#     search_results = sp.search(q=query, type='track', limit=50)  # Limitez le nombre de résultats ici

In [ ]:
import 